In [41]:
import numpy as np # always need it
import scipy as sp # often use it
import pylab as pl # do the plots
import seaborn as sns # for pretty plots
import matplotlib.pyplot as plt # for pretty plots

import ot # ot 

In [42]:
def Convolution_Wasserstein(a, b, C, l):
    u = np.ones(len(a))
    K = np.exp(-C/l)
    for i in range(len(a)):
        v = b / (K.T @ u)
        u = a / (K @ v)
    T = np.diag(u) @ K @ np.diag(v)
    return T

In [79]:
def Wasserstein_barycentre(mu_i, alpha, Ht, a, ent_sharp = False, N = 2):
    k = len(mu_i[0])
    d = np.zeros((2, k))
    v = np.ones((2,k))
    w = np.ones((2,k))
    for j in range(N):
        mu = np.ones(k)
        for i in range(2):
            w[i] = mu_i[i]/(Ht @ (a*v[i]))
            d[i] = v[i]*(Ht@(a*w[i]))
            mu = mu*d[i]**alpha[i]

        #entropic sharpening, optional
        if (ent_sharp):
            H_0 = H0(mu_i, a)
            mu = entropic_sharpening(mu, H_0, a)

        for i in range(2):
            v[i] = v[i]*mu/d[i]

    return mu




In [44]:
def entropic_sharpening(mu, H0, a):
    beta = 1
    if(H(mu, a) + np.transpose(a)*mu > H0 + 1 ):
        beta = sp.optimize.root(np.transpose(a)*mu**beta + H(mu**beta, a) - (1 + H0), beta > 0)
    return mu**beta

In [45]:
def H(mu, a):
    sum = 0
    for i in range(len(mu)):
        sum -= a[i] * mu[i] * np.log(mu[i])
    return sum

In [46]:
def H0(mu_i,a):
    for i in range(len(mu_i)):
        h = H(mu_i[i],a)
        if(h > H0):
            H0 = h 
    return h

In [47]:
def Wasserstein_propagation(V, E, V0, mu_V0, Ht, a, alpha, N = 200):
    m = len(V)
    ne = len(E)
    m0,k0 = np.shape(mu_V0)
    v = np.ones((ne,k0))
    w = np.ones((k,k0))
    mu = np.ones((m,k0))
    d = np.zeros((ne,k0))
    for j in range(N):
        for i in range(m):
            if (V[i] in V0):
                mu[i] = V0(V0.index(V[i]))
                for l in range(k): #on parcourt les aretes
                    e = E[l]
                    if e[1] == V[i] :
                        w[l] = mu / Ht @ (a * v[l])
                    if e[0] == V[i] :
                        v[l] = mu / Ht @ (a * w[l])
            else:
                omega = 0
                for l in range(k):
                    e = E[l]
                    if(e[1]==V[i] or e[0]==V[i]):
                        omega += alpha[l]
                for l in range(k): #on parcourt les aretes
                    e = E[l]
                    if e[1] == V[i] :
                        d[l] = w[l] * Ht @ (a * v[l])
                    if e[0] == V[i] :
                        d[l] = v[l] * Ht @ (a * w[l])
                    mu[i] = mu[i] * d[l]**(alpha[l]/omega)
                for l in range(k): #on parcourt les aretes
                    e = E[l]
                    if e[1] == V[i] :
                        w[l] = w[l] * mu[i]/d[l]
                    if e[0] == V[i] :
                        v[l] = v[l] * mu[i]/d[l]
            

    return mu

In [76]:
class point:
    def __init__(self, x, y, z, w):
        self.x = x
        self.y = y
        self.z = z
        self.weight = w

mesh1 = np.zeros((1000), point)
alpha1 = np.zeros(1000)

for i in range(10):
    for j in range(10):
        for k in range(10):
            mesh1[i*100+j*10+k] = point(i,j,k,1)
            alpha1[i*100+j*10+k] = 1

mesh2 = np.zeros(1000, point)
alpha2 = np.zeros(1000)
r = 5

for i in range(10):
    for j in range(10):
        for k in range(10):
            if np.linalg.norm([i - 5, j - 5.,k - 5.]) < r :
                mesh2[i*100+j*10+k] = point(i,j,k,1)
                alpha2[i*100+j*10+k] = 1
            else : 
                mesh2[i*100+j*10+k] = point(i,j,k,0)
                alpha2[i*100+j*10+k] = 0.1

alpha1 = alpha1/np.linalg.norm(alpha1)
alpha2 = alpha2/np.linalg.norm(alpha2)
print(alpha2)

[0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685
 0.00451685 0.00451685 0.00451685 0.00451685 0.00451685 0.0045

In [77]:
def kernel(n1, n2, gamma): 
    a = np.zeros((n1, n2))
    min = np.infty
    for i1 in range(n1):
        for i2 in range(n2):
            if min > -np.linalg.norm([mesh1[i1].x - mesh2[i2].x, mesh1[i1].y - mesh2[i2].y, mesh1[i1].z - mesh2[i2].z])**2 / gamma:
                min = -np.linalg.norm([mesh1[i1].x - mesh2[i2].x, mesh1[i1].y - mesh2[i2].y, mesh1[i1].z - mesh2[i2].z])**2 / gamma
            a[i1,i2] = np.exp(-np.linalg.norm([mesh1[i1].x/10 - mesh2[i2].x/10, mesh1[i1].y/10 - mesh2[i2].y/10, mesh1[i1].z/10 - mesh2[i2].z/10])**2 / gamma)
            # if a[i1,i2] == 0 : 
                # print(-np.linalg.norm([mesh1[i1].x - mesh2[i2].x, mesh1[i1].y - mesh2[i2].y, mesh1[i1].z - mesh2[i2].z])**2 / gamma)
                # return a
    print(min)
    return a

In [53]:
n1 = 1000
n2 = 1000
gamma = 0.1
Ht = kernel(n1, n2, gamma)
a = 1/(n1)

-2430.0


In [55]:
print(Ht[Ht==0])

[]


In [89]:
print(Wasserstein_barycentre((alpha1, alpha2), (1, 1), Ht, a, N=2))

[5.40064731e-06 1.42052551e-05 2.57086706e-05 3.55148019e-05
 4.13030625e-05 4.30825452e-05 4.02845648e-05 3.15677786e-05
 1.86450157e-05 7.42960183e-06 1.42052551e-05 3.87467798e-05
 7.23634147e-05 1.02347682e-04 1.20766416e-04 1.26731564e-04
 1.18397939e-04 9.22218803e-05 5.39504802e-05 2.12478055e-05
 2.57086706e-05 7.23634147e-05 1.38672799e-04 1.99774717e-04
 2.38322416e-04 2.51258140e-04 2.34709548e-04 1.82180081e-04
 1.05955712e-04 4.14236387e-05 3.55148019e-05 1.02347682e-04
 1.99774717e-04 2.91439849e-04 3.50200940e-04 3.70363465e-04
 3.46071892e-04 2.68197218e-04 1.55541711e-04 6.05850229e-05
 4.13030625e-05 1.20766416e-04 2.38322416e-04 3.50200940e-04
 4.22514111e-04 4.47610935e-04 4.18358517e-04 3.24010644e-04
 1.87682538e-04 7.29858188e-05 4.30825452e-05 1.26731564e-04
 2.51258140e-04 3.70363465e-04 4.47610935e-04 4.74491085e-04
 4.43406747e-04 3.43169491e-04 1.98571269e-04 7.71224137e-05
 4.02845648e-05 1.18397939e-04 2.34709548e-04 3.46071892e-04
 4.18358517e-04 4.434067